# Merging with external Datasets

The goal of this notebook is to merge the `movie_summaries.csv` file, we have made from the MovieSummaries Dataset in the `movie_summaries.ipynb` notebook, with external Datasets. The resulting Dataset will be enrich using GPT.

- [Description of the external Datasets](#external-datasets)
- [Data merging](#data-merging)
    - [TMDb](#tmdb)
    - [IMDb](#imdb)

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.utils.constants import *
from src.utils.helpers import *

## Description of the external Datasets

- [The Movies Dataset (TMDb)](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) contains metadata for 45,000 films listed in the Full MovieLens Dataset, all of which were released on or before July 2017. We will use this dataset, specifically the movies_metadata.csv file, primarily to enrich the movie.metadata.tsv of the CMU movie dataset. Additionally, this dataset includes plot keywords (in the keywords.csv file), which will be helpful for analyzing movie plots.

- [IMDB Non-Commercial Dataset](https://developer.imdb.com/non-commercial-datasets/) is a giant database containing over 50 Million movies from different regions, which allows us to complement our initial dataset specifically with movies created in the Soviet Union during the cold war. The CMU Movie Summaries data being largely focused on the United States.

## Data Merging

In [2]:
# Load the data
df_movies = pd.read_csv(DATA_FOLDER_PREPROCESSED + 'movie_summaries.csv')

### TMDb

In [3]:
# Load the data
df_tmdb_movies = pd.read_csv(
    TMDB_MOVIE,
    usecols=[
        "id",
        "title",
        "release_date",
        "revenue",
        "runtime",
        "genres",
        "production_countries",
        "original_language",
        "spoken_languages",
        "overview",
    ],
)

df_tmdb_keywords = pd.read_csv(TMDB_KEYWORDS)

In [4]:
# Use the helper function to preprocess the data before the merging
df_tmdb_movies = preprocess_movie_data(df_tmdb_movies)

# Merge the TMDb movies with TMDb keywords based on id
df_tmdb_movies["id"] = df_tmdb_movies["id"].astype(int)
df_tmdb_movies = pd.merge(df_tmdb_movies, df_tmdb_keywords, on="id", how="left")

for column_name in ["genres", "production_countries", "spoken_languages", "keywords"]:
    df_tmdb_movies[column_name] = df_tmdb_movies[column_name].apply(
        lambda row: (
            [item["name"] for item in ast.literal_eval(row)]
            if pd.notnull(row) and ast.literal_eval(row)
            else np.nan
        )
    )

# Merge with MoviesSummaries based on the title
df_merged_movies = pd.merge(
    df_movies,
    df_tmdb_movies,
    on="title",
    how="outer",
    suffixes=("_original", "_additional"),
)

df_merged_movies.head()

,wikipedia_id,freebase_id,title,release_date_original,revenue_original,runtime_original,languages,countries,genres_original,plot_summary,genres_additional,id,original_language,overview,production_countries,release_date_additional,revenue_additional,runtime_additional,spoken_languages,keywords
0,29988427.0,/m/0cryrj0,!Women Art Revolution,2010-09-01,NaN,83.0,['English Language'],"['United States of America', 'Canada']","['LGBT', 'History', 'Documentary']","!Women Art Revolution explores the ""secret his...",[Documentary],55245.0,en,"Through intimate interviews, provocative art, ...",NaN,2010-01-01 00:00:00,NaN,83.0,NaN,"[feminism, woman director]"
1,30332673.0,/m/0crs0hx,#1 Cheerleader Camp,2010-01-01,NaN,90.0,NaN,['United States of America'],"['Sports', 'Sex comedy', 'Comedy film', 'Comed...",Two horny college guys get summer jobs at a ch...,"[Comedy, Drama]",41371.0,en,A pair of horny college guys get summer jobs a...,[United States of America],2010-07-27 00:00:00,NaN,95.0,[English],[sport]
2,NaN,NaN,#Horror,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Drama, Mystery, Horror, Thriller]",301325.0,de,"Inspired by actual events, a group of 12 year ...",[United States of America],2015-11-20 00:00:00,NaN,90.0,"[English, Español]",NaN
3,NaN,NaN,#Horror,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Drama, Mystery, Horror, Thriller]",301325.0,de,"Inspired by actual events, a group of 12 year ...",[United States of America],2015-11-20 00:00:00,NaN,90.0,"[English, Español]",NaN
4,NaN,NaN,#chicagoGirl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Documentary],267752.0,en,From her childhood bedroom in the Chicago subu...,NaN,2013-11-21 00:00:00,NaN,74.0,NaN,NaN


We observe some duplicated columns, let's combine them.

In [5]:
for _, column_name in enumerate(df_merged_movies.columns):

    if column_name in ["release_date_original", "revenue_original", "runtime_original"]:
        # craft the additional column name
        new_column_name = column_name[: -len("_original")]
        column_name_additional = new_column_name + "_additional"
        # fill the missing values
        df_merged_movies[new_column_name] = df_merged_movies[column_name].combine_first(
            df_merged_movies[column_name_additional]
        )
        # drop the original and additional column
        df_merged_movies = df_merged_movies.drop(
            columns=[column_name, column_name_additional]
        )

df_merged_movies["release_date"] = pd.to_datetime(
    df_merged_movies["release_date"], errors="coerce"
)

In [6]:
df_merged_movies = combine_columns(df_merged_movies, "genres_original", "genres_additional")
df_merged_movies = df_merged_movies.rename(columns={"genres_original": "genres"})

df_merged_movies = combine_columns(df_merged_movies, "languages", "spoken_languages")

df_merged_movies = combine_columns(df_merged_movies, "countries", "production_countries")

df_merged_movies = df_merged_movies.drop(
    columns=["spoken_languages", "production_countries", "id", "genres_additional"]
)

In [7]:
df_merged_movies.head()

,wikipedia_id,freebase_id,title,languages,countries,genres,plot_summary,original_language,overview,keywords,release_date,revenue,runtime
0,29988427.0,/m/0cryrj0,!Women Art Revolution,NaN,NaN,[Documentary],"!Women Art Revolution explores the ""secret his...",en,"Through intimate interviews, provocative art, ...","[feminism, woman director]",2010-09-01,NaN,83.0
1,30332673.0,/m/0crs0hx,#1 Cheerleader Camp,[English],[United States of America],"[Comedy, Drama]",Two horny college guys get summer jobs at a ch...,en,A pair of horny college guys get summer jobs a...,[sport],2010-01-01,NaN,90.0
2,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,de,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0
3,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,de,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0
4,NaN,NaN,#chicagoGirl,NaN,NaN,[Documentary],NaN,en,From her childhood bedroom in the Chicago subu...,NaN,2013-11-21,NaN,74.0


### IMDb

In [8]:
# Load the data
title_akas = pd.read_csv(IMDB_AKA, sep="\t", usecols=["titleId", "title", "region"])
title_basics = pd.read_csv(
    IMDB_BASIC,
    sep="\t",
    usecols=["tconst", "primaryTitle", "titleType", "startYear", "genres"],
)

In [9]:
# Merge the DataFrames on the common column tconst
imdb_movies = pd.merge(title_akas, title_basics, left_on="titleId", right_on="tconst")

# Select only the columns we need and rename `titleId` to `tconst` for consistency
imdb_movies = imdb_movies[
    ["tconst", "title", "primaryTitle", "region", "titleType", "startYear", "genres"]
]

# select region that are related to the Soviet Union
regions = [
    "SU",
    "RU",
    "UA",
    "BY",
    "KZ",
    "UZ",
    "GE",
    "AM",
    "AZ",
    "LT",
    "LV",
    "EE",
    "TM",
    "KG",
    "TJ",
    "MD",
]

# Filter the movies that are related to the Soviet Union
soviet_movies = imdb_movies[
    imdb_movies["region"].fillna("").str.contains("|".join(regions), case=False)
]

# Drop the columns that are not needed
soviet_movies = (
    soviet_movies[soviet_movies["titleType"] == "movie"]
    .drop_duplicates(subset="primaryTitle", keep="first")
    .drop(columns=["title", "titleType"])
)

soviet_movies.head()

,tconst,primaryTitle,region,startYear,genres
762,tt0000147,The Corbett-Fitzsimmons Fight,RU,1897,"Documentary,News,Sport"
4507,tt0001338,A Night in May,RU,1910,Drama
4880,tt0001475,Amor fatal,SUHH,1911,"Drama,Romance"
5977,tt0001892,Den sorte drøm,RU,1911,Drama
6183,tt0001964,The Traitress,RU,1911,Drama


We use the IMDb API to get the plot of the movies. We commented out the code as it takes a long time to run.

In [10]:
# from concurrent.futures import ThreadPoolExecutor
# from tqdm import tqdm
# import time
# from imdb import IMDb
# from functools import partial


# time.sleep(1)

# imdb_instance = IMDb()

# # Use partial to pass the IMDb instance to the function
# get_plot_with_imdb = partial(get_plot_summary, imdb_instance=imdb_instance)

# with ThreadPoolExecutor(max_workers=4) as executor:

#     soviet_movies["plot"] = list(
#         tqdm(
#             executor.map(get_plot_with_imdb, soviet_movies["tconst"]),
#             total=len(soviet_movies),
#         )
#     )

# save the dataframe to soviet_movies.tsv
# soviet_movies.to_csv(DATA_FOLDER_PREPROCESSED + "soviet_movies.tsv", sep="\t")

In [22]:
soviet_movies = pd.read_csv(
    DATA_FOLDER_PREPROCESSED + "soviet_movies.tsv",
    sep="\t",
    usecols=[
        "tconst",
        "title",
        "primaryTitle",
        "region",
        "titleType",
        "startYear",
        "genres",
        "plot"
    ],
)

print(soviet_movies.columns)


Index(['tconst', 'title', 'primaryTitle', 'region', 'titleType', 'startYear',
       'genres', 'plot'],
      dtype='object')


In [12]:
# Drop the columns that are not needed and rename the columns to match our column names
soviet_movies = soviet_movies.drop(columns=["title", "titleType"]).rename(
    columns={"primaryTitle": "title", "startYear": "release_date", "plot":"plot_summary"}
)

soviet_movies["release_date"] = pd.to_datetime(
    soviet_movies["release_date"], format="%Y", errors="coerce"
)

soviet_movies["genres"] = soviet_movies["genres"].apply(
    lambda genres: genres.split(",")
)
soviet_movies.head()

,tconst,title,region,release_date,genres,plot_summary
0,tt0001338,A Night in May,RU,1910-01-01,[Drama],NaN
1,tt0001475,Amor fatal,SUHH,1911-01-01,"[Drama, Romance]",NaN
2,tt0001892,Den sorte drøm,RU,1911-01-01,[Drama],Two men of high rank are both wooing the beaut...
3,tt0001964,The Traitress,RU,1911-01-01,[Drama],A woman betrays the regiment location in which...
4,tt0002022,Anfisa,RU,1912-01-01,[Drama],The Kostomarov family falls apart when a femal...


In [13]:
# Merge the ´soviet_movies´ dataframe with the ´df_merged_movies´
df_merged_movies = pd.merge(df_merged_movies, soviet_movies, on="title", how="outer", suffixes=("_original", "_additional"))
df_merged_movies.head()

,wikipedia_id,freebase_id,title,languages,countries,genres_original,plot_summary_original,original_language,overview,keywords,release_date_original,revenue,runtime,tconst,region,release_date_additional,genres_additional,plot_summary_additional
0,29988427.0,/m/0cryrj0,!Women Art Revolution,NaN,NaN,[Documentary],"!Women Art Revolution explores the ""secret his...",en,"Through intimate interviews, provocative art, ...","[feminism, woman director]",2010-09-01,NaN,83.0,NaN,NaN,NaT,NaN,NaN
1,30332673.0,/m/0crs0hx,#1 Cheerleader Camp,[English],[United States of America],"[Comedy, Drama]",Two horny college guys get summer jobs at a ch...,en,A pair of horny college guys get summer jobs a...,[sport],2010-01-01,NaN,90.0,NaN,NaN,NaT,NaN,NaN
2,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,de,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0,NaN,NaN,NaT,NaN,NaN
3,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,de,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0,NaN,NaN,NaT,NaN,NaN
4,NaN,NaN,#chicagoGirl,NaN,NaN,[Documentary],NaN,en,From her childhood bedroom in the Chicago subu...,NaN,2013-11-21,NaN,74.0,NaN,NaN,NaT,NaN,NaN


Again, we have some duplicated columns.

In [14]:
df_merged_movies["release_date_original"] = df_merged_movies["release_date_original"].combine_first(
    df_merged_movies["release_date_additional"]
)

df_merged_movies = combine_columns(df_merged_movies, "genres_original", "genres_additional")


df_merged_movies = df_merged_movies.rename(
    columns={"release_date_original": "release_date", "genres_original": "genres"}
)
df_merged_movies = df_merged_movies.drop(columns=["release_date_additional", "genres_additional"])

In [15]:
def clean_languages(row):
    if isinstance(row['languages'], list):
        languages = set([string.replace(" Language", "") for string in row['languages']])
    else:
        languages = set([row['original_language']]) if isinstance(row['original_language'], str) else np.nan
    
    return languages

In [16]:
df_merged_movies['languages'] = df_merged_movies.apply(clean_languages, axis=1)

# Map ISO language codes to their usual names
mapping = {
    'en': 'English', 'de': 'German', 'it': 'Italian', 'hi': 'Hindi', 'zh': 'Chinese', 
    'fr': 'French', 'ko': 'Korean', 'ja': 'Japanese', 'nl': 'Dutch', 'te': 'Telugu', 
    'sv': 'Swedish', 'bs': 'Bosnian', 'es': 'Spanish', 'cn': 'Chinese', 'no': 'Norwegian', 
    'is': 'Icelandic', 'pl': 'Polish', 'ru': 'Russian', 'ro': 'Romanian', 'th': 'Thai', 
    'ab': 'Abkhazian', 'et': 'Estonian', 'fi': 'Finnish', 'el': 'Greek', 'ta': 'Tamil', 
    'pt': 'Portuguese', 'ur': 'Urdu', 'fa': 'Persian', 'da': 'Danish', 'tr': 'Turkish', 
    'nb': 'Norwegian Bokmål', 'xx': 'Unknown', 'sl': 'Slovenian', 'pa': 'Punjabi', 
    'sr': 'Serbian', 'sh': 'Serbo-Croatian', 'hu': 'Hungarian', 'lv': 'Latvian', 
    'cs': 'Czech', 'bn': 'Bengali', 'uk': 'Ukrainian', 'sq': 'Albanian', 'he': 'Hebrew', 
    'ml': 'Malayalam', 'vi': 'Vietnamese', 'mr': 'Marathi', 'ar': 'Arabic', 'ay': 'Aymara', 
    'ms': 'Malay', 'ka': 'Georgian', 'id': 'Indonesian', 'hr': 'Croatian', 'bg': 'Bulgarian', 
    'mk': 'Macedonian', 'bm': 'Bambara', 'tl': 'Tagalog', 'ku': 'Kurdish', 'ca': 'Catalan', 
    'sk': 'Slovak', 'uz': 'Uzbek', 'wo': 'Wolof', 'lo': 'Lao', 'gl': 'Galician', 'fy': 'Frisian', 
    'lt': 'Lithuanian', 'eu': 'Basque', 'am': 'Amharic', 'cy': 'Welsh', 'eo': 'Esperanto', 
    'kk': 'Kazakh', 'qu': 'Quechua', 'kn': 'Kannada', 'ne': 'Nepali', 'iu': 'Inuktitut', 
    'bo': 'Tibetan', 'rw': 'Kinyarwanda', 'jv': 'Javanese', 'ps': 'Pashto', 'ky': 'Kyrgyz', 
    'af': 'Afrikaans', 'la': 'Latin', 'mt': 'Maltese', 'hy': 'Armenian', 'mn': 'Mongolian', 
    'si': 'Sinhalese', 'sm': 'Samoan', 'lb': 'Luxembourgish', 'tg': 'Tajik', 'zu': 'Zulu'
}

df_merged_movies['languages'] = df_merged_movies['languages'].apply(lambda bag: 
    [mapping.get(string, string) for string in bag] if isinstance(bag, set) else bag)

df_merged_movies = df_merged_movies.drop('original_language', axis=1)
df_merged_movies.head()

,wikipedia_id,freebase_id,title,languages,countries,genres,plot_summary_original,overview,keywords,release_date,revenue,runtime,tconst,region,plot_summary_additional
0,29988427.0,/m/0cryrj0,!Women Art Revolution,[English],NaN,[Documentary],"!Women Art Revolution explores the ""secret his...","Through intimate interviews, provocative art, ...","[feminism, woman director]",2010-09-01,NaN,83.0,NaN,NaN,NaN
1,30332673.0,/m/0crs0hx,#1 Cheerleader Camp,[English],[United States of America],"[Comedy, Drama]",Two horny college guys get summer jobs at a ch...,A pair of horny college guys get summer jobs a...,[sport],2010-01-01,NaN,90.0,NaN,NaN,NaN
2,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0,NaN,NaN,NaN
3,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0,NaN,NaN,NaN
4,NaN,NaN,#chicagoGirl,[English],NaN,[Documentary],NaN,From her childhood bedroom in the Chicago subu...,NaN,2013-11-21,NaN,74.0,NaN,NaN,NaN


In [17]:
def clean_countries(row):
    if isinstance(row['countries'], list):
        countries = set(row['countries'])
    else:
        countries = set([row['region']]) if isinstance(row['region'], str) else np.nan
    
    return countries

In [18]:
df_merged_movies['countries'] = df_merged_movies.apply(clean_countries, axis=1)

region_to_country = {
    "SU": "Soviet Union",
    "SUHH": "Soviet Union",
    "RU": "Russia",
    "UA": "Ukraine",
    "BY": "Belarus",
    "KZ": "Kazakhstan",
    "UZ": "Uzbekistan",
    "GE": "Georgia",
    "AM": "Armenia",
    "AZ": "Azerbaijan",
    "LT": "Lithuania",
    "LV": "Latvia",
    "EE": "Estonia",
    "TM": "Turkmenistan",
    "KG": "Kyrgyzstan",
    "TJ": "Tajikistan",
    "MD": "Moldova"
}

df_merged_movies['countries'] = df_merged_movies['countries'].apply(lambda bag: 
    [region_to_country.get(string, string) for string in bag] if isinstance(bag, set) else bag)

df_merged_movies = df_merged_movies.drop('region', axis=1)
df_merged_movies.head()

,wikipedia_id,freebase_id,title,languages,countries,genres,plot_summary_original,overview,keywords,release_date,revenue,runtime,tconst,plot_summary_additional
0,29988427.0,/m/0cryrj0,!Women Art Revolution,[English],NaN,[Documentary],"!Women Art Revolution explores the ""secret his...","Through intimate interviews, provocative art, ...","[feminism, woman director]",2010-09-01,NaN,83.0,NaN,NaN
1,30332673.0,/m/0crs0hx,#1 Cheerleader Camp,[English],[United States of America],"[Comedy, Drama]",Two horny college guys get summer jobs at a ch...,A pair of horny college guys get summer jobs a...,[sport],2010-01-01,NaN,90.0,NaN,NaN
2,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0,NaN,NaN
3,NaN,NaN,#Horror,"[English, Español]",[United States of America],"[Mystery, Thriller, Drama, Horror]",NaN,"Inspired by actual events, a group of 12 year ...",NaN,2015-11-20,NaN,90.0,NaN,NaN
4,NaN,NaN,#chicagoGirl,[English],NaN,[Documentary],NaN,From her childhood bedroom in the Chicago subu...,NaN,2013-11-21,NaN,74.0,NaN,NaN


In [19]:
df_merged_movies['plot_summary'] = df_merged_movies['plot_summary_original'].combine_first(df_merged_movies['overview']).combine_first(df_merged_movies['plot_summary_additional'])

# Dropping the used columns
df_merged_movies.drop(columns=['plot_summary_original', 'overview', 'plot_summary_additional', 'tconst', 'revenue'], inplace=True)

# drop nan values
df_merged_movies = df_merged_movies.dropna(subset=['plot_summary', 'title', 'release_date', 'countries'])

df_merged_movies = df_merged_movies[df_merged_movies['release_date'] <= pd.to_datetime('1995')]
df_merged_movies = df_merged_movies[df_merged_movies['release_date'] >= pd.to_datetime('1945')]

df_merged_movies["release_date"] = df_merged_movies["release_date"].dt.year

df_merged_movies.drop_duplicates(subset=['title', 'release_date'], inplace=True)

In [20]:
df_merged_movies.head()

,wikipedia_id,freebase_id,title,languages,countries,genres,keywords,release_date,runtime,plot_summary
5,4213160.0,/m/0bq8q8,$,NaN,[Soviet Union],"[Crime, Comedy, Drama]",NaN,1971,119.0,"Set in Hamburg, West Germany, several criminal..."
6,NaN,NaN,"$1,000 on the Black","[Italiano, Deutsch]","[Italy, Germany]",[Western],NaN,1966,104.0,Johnny Liston has just been released from pris...
7,NaN,NaN,"$10,000 Blood Money",NaN,[Soviet Union],"[Western, Drama]",NaN,1967,NaN,Hired by a Mexican landowner to rescue his dau...
8,NaN,NaN,"$100,000 for Ringo",[Italiano],[Italy],"[Drama, Western]","[spaghetti western, whipping]",1965,98.0,A stranger rides into Rainbow Valley where he'...
18,NaN,NaN,'Anna' i wampir,NaN,[Soviet Union],[Crime],NaN,1982,NaN,"Silesia in Poland, late 60s. Bodies of vicious..."


In [21]:
df_merged_movies.to_csv(DATA_FOLDER_PREPROCESSED + "merged_movies.csv", index=False)